## Rolling Windows Function

### **Inputs:**
- `returns`: pandas DataFrame of asset returns with dates as index and assets as columns.
- `memory`: window size (number of periods).
- `min_period`: minimum periods required to compute a covariance matrix, default is 20.

### **Breakdown of Variables:**
- `min_periods`: Ensures periods are at least 1 or 20 to avoid division by 0.
- `times`: Stores the timestamp of each row of returns.
- `assets`: Stores asset names for each column of returns.
- `returns`: Converts the DataFrame into a numpy array (for faster computing).
- `Sigmas`: A 3D array to store the covariance matrices at each time step (# time step, # assets, # assets).
- `Sigmas[0]`: Initializes the first covariance matrix as the outer product of the first return vector.

### **Logic + Useful Formulas to Add to PowerPoint:**
To compute rolling window covariance matrices over time using recursive updates for both increasing and fixed window sizes:

### 1. Initialize
Start covariance matrix computed from the first observation:

$\Sigma_{0} = r_{0} r_{0}^{\top}$

### 2. Recursive Updates
##### a. Scaling factors adjusted based on the number of observations:
- 
##### b. Before the window reaches full size $(t < \text{memory})$:
- Let $S_t$ be the cumulative sum up to time $t$:

  $S_{t} = S_{t-1} + r_{t} r_{t}^{\top}$

- A sample covariance matrix at time $t$ is:

  $\Sigma_{t} = \frac{1}{t+1} S_{t}$

- Update the cumulative covariance with each new observation:

  $\Sigma_{t} = \left(\frac{t}{t+1}\right) \Sigma_{t-1} + \frac{1}{t+1} r_{t} r_{t}^{\top}$

- Since $\Sigma_{t-1} = \frac{1}{t} S_{t-1}$, adjust for the increasing number of observations.

##### c. After the window reaches full size $(t > \text{memory})$:
- Maintain a rolling sum by adding the new observation and subtracting the oldest one:

  $S_{t} = S_{t-1} + r_{t} r_{t}^{\top} - r_{t-\text{memory}} r_{t-\text{memory}}^{\top}$

- Sample covariance matrix, keeping the window size fixed at `memory` (e.g., 125 days):

  $\Sigma_{t} = \frac{1}{\text{memory}} S_{t}$

- Recursive formula combining the two:

  $\Sigma_{t} = \Sigma_{t-1} + \frac{1}{\text{memory}} \left(r_{t} r_{t}^{\top} - r_{t-\text{memory}} r_{t-\text{memory}}^{\top}\right)$

#### 3. Return
Return a dictionary mapping each timestamp to its corresponding covariance matrix.

### **How Scaling Factors are Derived**

#### 1. Express $\Sigma_t$ in Terms of $\Sigma_{t-1}$

First, let's express $S_t$ in terms of $\Sigma_{t-1}$:

$
S_t = S_{t-1} + r_t r_t^{\top} = N_{t-1} \Sigma_{t-1} + r_t r_t^{\top}
$

Since $S_{t-1} = N_{t-1} \Sigma_{t-1}$.

#### 2. Substitute $S_t$ into $\Sigma_t$

$
\Sigma_t = \frac{1}{N_t} (N_{t-1} \Sigma_{t-1} + r_t r_t^{\top})
$

#### 3. Simplify the Expression

Recognize that $N_t = N_{t-1} + 1$, so:

$
\Sigma_t = \left(\frac{N_{t-1}}{N_t}\right) \Sigma_{t-1} + \left(\frac{1}{N_t}\right) r_t r_t^{\top}
$

In terms of $\alpha_{\text{old}}$ and $\alpha_{\text{new}}$:

$
\alpha_{\text{old}} = \frac{1}{N_{t-1}}, \quad \alpha_{\text{new}} = \frac{1}{N_t}
$

#### 4. Thus, the recursive update formula is:

$
\Sigma_t = \left(\frac{\alpha_{\text{old}}}{\alpha_{\text{new}}}\right) \Sigma_{t-1} + \alpha_{\text{new}} r_t r_t^{\top}
$

Therefore:

$
\alpha_{\text{old}} = \frac{1}{t+1}, \quad \alpha_{\text{new}} = \frac{1}{t+2}
$


In [ ]:
def rolling_window(returns, memory, min_periods=20):
    min_periods = max(min_periods, 1)

    times = returns.index
    assets = returns.columns

    returns = returns.values

    Sigmas = np.zeros((returns.shape[0], returns.shape[1], returns.shape[1]))
    Sigmas[0] = np.outer(returns[0], returns[0])

    for t in range(1, returns.shape[0]):
        alpha_old = 1 / min(t + 1, memory)
        alpha_new = 1 / min(t + 2, memory)

        if t >= memory:
            Sigmas[t] = alpha_new / alpha_old * Sigmas[t - 1] + alpha_new * (
                np.outer(returns[t], returns[t])
                - np.outer(returns[t - memory], returns[t - memory])
            )
        else:
            Sigmas[t] = alpha_new / alpha_old * Sigmas[t - 1] + alpha_new * (
                np.outer(returns[t], returns[t])
            )

    Sigmas = Sigmas[min_periods - 1 :]
    times = times[min_periods - 1 :]

    return {
        times[t]: pd.DataFrame(Sigmas[t], index=assets, columns=assets)
        for t in range(len(times))
    }

In [ ]:
def _ewma_cov(data, halflife, min_periods=0):
    """
    param data: Txn pandas DataFrame of returns
    param halflife: float, halflife of the EWMA
    """

    for t, ewma in _general(
        data.values,
        times=data.index,
        halflife=halflife,
        fct=lambda x: np.outer(x, x),
        min_periods=min_periods,
    ):
        if not np.isnan(ewma).all():
            yield t, pd.DataFrame(index=data.columns, columns=data.columns, data=ewma)


In [ ]:

def iterated_ewma(
    returns,
    vola_halflife,
    cov_halflife,
    min_periods_vola=20,
    min_periods_cov=20,
    mean=False,
    mu_halflife1=None,
    mu_halflife2=None,
    clip_at=None,
    nan_to_num=True,
):
    """
    param returns: pandas dataframe with returns for each asset
    param vola_halflife: half life for volatility
    param cov_halflife: half life for covariance
    param min_preiods_vola: minimum number of periods to use for volatility
    ewma estimate
    param min_periods_cov: minimum number of periods to use for covariance
    ewma estimate
    param mean: whether to estimate mean; if False, assumes zero mean data
    param mu_halflife1: half life for the first mean adjustment; if None, it is
    set to vola_halflife; only applies if mean=True
    param mu_halflife2: half life for the second mean adjustment; if None, it is
    set to cov_halflife; only applies if mean=True
    param clip_at: winsorizes ewma update at +-clip_at*(current ewma) in ewma;
    if None, no winsorization is performed
    nan_to_num: if True, replace NaNs in returns with 0.0
    """
    mu_halflife1 = mu_halflife1 or vola_halflife
    mu_halflife2 = mu_halflife2 or cov_halflife

In [ ]:
def MSE(returns, covariances):
    returns_shifted = returns.shift(-1)

    MSEs = []
    for time, cov in covariances.items():
        realized_cov = returns_shifted.loc[time].values.reshape(
            -1, 1
        ) @ returns_shifted.loc[time].values.reshape(1, -1)
        MSEs.append(np.linalg.norm(cov - realized_cov) ** 2)

    return pd.Series(MSEs, index=covariances.keys())

In [ ]:
def log_likelihood_low_rank(returns, Sigmas, means=None):
    """
    Computes the log-likelihoods

    param returns: pandas DataFrame of returns
    param Sigmas: dictionary of covariance matrices where each covariance matrix
                  is a namedtuple with fields "F" and "d"
    param means: pandas DataFrame of means

    Note: Sigmas[time] is covariance prediction for returns[time+1]
        same for means.loc[time]
    """
    returns = returns.shift(-1)

    ll = []
    m = np.zeros_like(returns.iloc[0].values).reshape(-1, 1)

    times = []

    for time, low_rank in Sigmas.items():
        # TODO: forming the covariance matrix is bad...
        cov = low_rank.F @ (low_rank.F).T + np.diag(low_rank.d)

        if not returns.loc[time].isna()[0]:
            if means is not None:
                m = means.loc[time].values.reshape(-1, 1)
            ll.append(
                _single_log_likelihood(
                    returns.loc[time].values.reshape(-1, 1), cov.values, m
                )
            )
            times.append(time)

    return pd.Series(ll, index=times).astype(float)

In [ ]:
def ecdf(data):
    n = len(data)
    x = np.sort(data)
    y = np.arange(1, n + 1) / n
    return x, y

In [ ]:
def add_to_diagonal(Sigmas, lamda):
    """
    Adds lamda*diag(Sigma) to each covariance (Sigma) matrix in Sigmas

    param Sigmas: dictionary of covariance matrices
    param lamda: scalar
    """
    for key in Sigmas.keys():
        Sigmas[key] = Sigmas[key] + lamda * np.diag(np.diag(Sigmas[key]))

    return Sigmas

In [ ]:
def from_row_to_covariance(row, n):
    """
    Convert upper diagonal part of covariance matrix to a covariance matrix
    """
    Sigma = np.zeros((n, n))

    # set upper triangular part
    upper_mask = np.triu(np.ones((n, n)), k=0).astype(bool)
    Sigma[upper_mask] = row

    # set lower triangular part
    lower_mask = np.tril(np.ones((n, n)), k=0).astype(bool)
    Sigma[lower_mask] = Sigma.T[lower_mask]
    return Sigma